In [1]:
from ignite.contrib.metrics.regression import *
from ignite.contrib.metrics import *
from ignite.handlers import *
from ignite.metrics import *
from ignite.engine import *
from ignite.utils import *

from torch.utils.data import DataLoader
from torch.autograd import Variable
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data
import torch.nn as nn

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from collections import OrderedDict
from scipy import io
import pandas as pd
import numpy as np
import datetime
import torch
import time
import pdb
import os
import gc

print(datetime.datetime.now(), "model.py code start")

BATCH_SIZE = 107
EPOCH = 900
LEARNING_RATE = 0.2
ANNEALING_RATE = 0.999
HIDDEN_UNITS = [180, 200, 250, 80, 100, 120]
K_FOLD = 1
MAT_PATH = "C:/Users/HILAB_Labtop_02/Desktop/insung/ecg-dbn/data/mit.mat"

2022-05-20 12:14:23.197519 model.py code start


In [2]:
device = torch.device('cuda')
print(torch.cuda.get_device_name(device))
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

NVIDIA GeForce RTX 3080 Laptop GPU


## RBM set-up as object

In [3]:
class RBM(nn.Module): 
    with torch.cuda.device(0):
        def __init__(self, n_vis, n_hid, k, batch):
            super(RBM, self).__init__()
            self.W      = nn.Parameter(torch.randn(1, batch, device=device) * 1e-2)
            self.n_vis  = n_vis
            self.n_hid  = n_hid
            self.k      = k
            self.batch  = batch
            self.v_bias = nn.Parameter(torch.zeros(n_vis, 1, device=device))
            self.h_bias = nn.Parameter(torch.zeros(n_hid, 1, device=device))
        
        def sample_from_p(self, p):
            return F.relu(
                torch.sign(
                    p - Variable(torch.randn(p.size(), device=device))
                )
            ).to(device=device)

        ''' ISSUE PART '''
        def v_to_h(self, v):
            w = (self.W.clone())

            p_h = F.sigmoid(
                F.linear(v, w)
            ).to(device=device)

            sample_h = self.sample_from_p(p_h)
            return p_h, sample_h

        def h_to_v(self, h):
            w = self.W.t().clone()

            p_v = F.sigmoid(
                F.linear(h, w)
            ).to(device=device)

            sample_v = self.sample_from_p(p_v)
            return p_v, sample_v
        
        def forward(self, v):
            pre_h1, h1 = self.v_to_h(v)
            h_ = h1

            for _ in range(self.k):
                pre_v_, v_ = self.h_to_v(h_)
                pre_h_, h_ = self.v_to_h(v_)
            return v, v_
        
        def get_weight(self):
            return self.W

In [4]:
class SVM(nn.Module):
    with torch.cuda.device(0):
        def __init__(self, lr, n_x):
            super(SVM, self).__init__()
            self.lr = lr
            self.fully = nn.Linear(n_x, 1).to(device=device)
        
        def forward(self, x):
            fwd = self.fully(x)
            return fwd

## Init of custom functions

In [5]:
def eval_step(engine, batch):
    return batch

default_model = nn.Sequential(OrderedDict([
    ('base', nn.Linear(4, 2)),
    ('fc', nn.Linear(2, 1))
]))

default_evaluator = Engine(eval_step)

def get_acc(y_true, y_pred):
    metric = Accuracy()
    metric.attach(default_evaluator, "accuracy")
    state = default_evaluator.run([[y_pred, y_true]])
    return state.metrics["accuracy"]

In [6]:
def to_categorical(y, num_classes):
    return np.eye(num_classes, dtype='uint8')[y]

## Pre-processing

In [7]:
mit = io.loadmat(MAT_PATH)
mit = mit['mit']
df_mit = pd.DataFrame(data = mit)

print(df_mit[428].value_counts(), '\n')
# 0 = N
# 1 = Q
# 2 = S
# 3 = V
# 4 = F

Y = np.array(df_mit[428].values).astype(np.int8)
X = np.array(df_mit[list(range(428))].values)[..., np.newaxis]

oneHot = LabelEncoder()
oneHot.fit(Y)
Y = oneHot.transform(Y)

X = X.reshape(-1, 428, 1)
Y = to_categorical(Y, 5)

X_train, X_val, Y_train, Y_val = train_test_split(X, Y,
                                                test_size=0.3,
                                                random_state=42,
                                                stratify=Y)

print("X_train shape: ", X_train.shape)
print("Y_train shape: ", Y_train.shape)
print("X_val shape: ", X_val.shape)
print("Y_val shape: ", Y_val.shape)

0.0    89921
3.0     6999
2.0     2777
4.0      801
1.0       15
Name: 428, dtype: int64 

X_train shape:  (70359, 428, 1)
Y_train shape:  (70359, 5)
X_val shape:  (30154, 428, 1)
Y_val shape:  (30154, 5)


In [8]:
train_dataloader = DataLoader(X_train,
                              batch_size=BATCH_SIZE,
                              num_workers=0,
                              shuffle=True)

test_dataloader = DataLoader(Y,
                             batch_size=BATCH_SIZE,
                             num_workers=0,
                             shuffle=True)

## Call models, probabilties.

In [9]:
bbrbm_first = RBM(n_vis=187, n_hid=HIDDEN_UNITS[0], k=K_FOLD, batch=BATCH_SIZE).to(device=device)
bbrbm_second = RBM(n_vis=187, n_hid=HIDDEN_UNITS[1], k=K_FOLD, batch=BATCH_SIZE).to(device=device)
bbrbm_third = RBM(n_vis=187, n_hid=HIDDEN_UNITS[2], k=K_FOLD, batch=BATCH_SIZE).to(device=device)

gbrbm_first = RBM(n_vis=187, n_hid=HIDDEN_UNITS[3], k=K_FOLD, batch=BATCH_SIZE).to(device=device)
gbrbm_second = RBM(n_vis=187, n_hid=HIDDEN_UNITS[4], k=K_FOLD, batch=BATCH_SIZE).to(device=device)
gbrbm_third = RBM(n_vis=187, n_hid=HIDDEN_UNITS[5], k=K_FOLD, batch=BATCH_SIZE).to(device=device)

first_train_op = optim.Adagrad(bbrbm_first.parameters(), LEARNING_RATE)
second_train_op = optim.Adagrad(bbrbm_second.parameters(), LEARNING_RATE)
third_train_op = optim.Adagrad(bbrbm_third.parameters(), LEARNING_RATE)

gb_first_train_op = optim.Adagrad(gbrbm_first.parameters(), LEARNING_RATE)
gb_second_train_op = optim.Adagrad(gbrbm_second.parameters(), LEARNING_RATE)
gb_third_train_op = optim.Adagrad(gbrbm_third.parameters(), LEARNING_RATE)

omse_loss = list()
output_gb = list()
best_acc = float()
svm_best_acc = float()
mse_loss = nn.MSELoss()

# gaussian_std = torch.arange(1, 0, -0.00537, device=device)
gaussian_std = torch.arange(1, 0, -0.0094, device=device)
print(gaussian_std.size())

svm_model = SVM(lr=LEARNING_RATE, n_x=107)
svm_optimizer = optim.Adagrad(svm_model.parameters(), lr=LEARNING_RATE)

torch.Size([107])


## Train Model
Always careful at this part. Some of setting are can be occure system errors like cuda stuff.

In [10]:
'''BBRBM Train Part'''

loss_ = []
output_bb = []
print("RBM START!")

for epoch in range(EPOCH):
    tmp_acc = float()
    run_acc = float()
    start = time.time()
    '''First bbrbm'''
    for i, (data) in enumerate(train_dataloader):
        if i == 10051:
            break

        data = Variable(torch.tensor(data.uniform_(0, 1), dtype=torch.float32))
        
        sample_data = torch.bernoulli(data).view(-1, BATCH_SIZE).to(device=device)
        fs_data = sample_data
        
        # tensor binary
        fvog_first, v1 = bbrbm_first(sample_data)
        omse_loss = mse_loss(fvog_first, v1)
        
        first_train_op.zero_grad()
        first_train_op.step()
        omse_loss.backward()
    
    for _, (data) in enumerate(v1): 
        data = Variable(
                torch.tensor(data, dtype=torch.float32)
        ).uniform_(0, 1)

        sample_data = torch.bernoulli(data).view(-1, BATCH_SIZE).to(device=device)

        # tensor binary
        vog_second, v2 = bbrbm_second(sample_data)
        omse_loss = mse_loss(vog_second, v2)

        second_train_op.zero_grad()
        omse_loss.backward()
        second_train_op.step()
    
    for _, (data) in enumerate(v2):
        start = time.time()
        data = Variable(
                torch.tensor(data, dtype=torch.float32)
        ).uniform_(0, 1)

        sample_data = torch.bernoulli(data).view(-1, BATCH_SIZE).to(device=device)

        vog_third, v3 = bbrbm_third(sample_data)
        omse_loss = mse_loss(vog_third, v3)
        
        third_train_op.zero_grad()
        omse_loss.backward()
        third_train_op.step()

        run_acc += (sample_data == v3).sum().item()
    
    '''
        GBRBM GBRBM GBRBM GBRBM GBRBM GBRBM GBRBM 
    '''

    for i, (data) in enumerate(output_bb):
        data = Variable(
                torch.tensor(data, dtype=torch.float32)
        ).uniform_(0, 1)
        
        sample_data = torch.normal(mean=data, std=gaussian_std).to(device=device)

        # tensor binary
        vog_first, v1 = gbrbm_first(sample_data)
        omse_loss = mse_loss(vog_first, v1)

        gb_first_train_op.zero_grad()
        gb_first_train_op.step()
        omse_loss.backward()

    for _, (data) in enumerate(v1): 
        data = Variable(
                torch.tensor(data, dtype=torch.float32)
        ).uniform_(0, 1)

        # sample_data = torch.normal(mean=data, std=gaussian_std).view(-1, BATCH_SIZE).to(device=device)
        sample_data = torch.normal(mean=data, std=gaussian_std).to(device=device)

        # tensor binary
        vog_second, v2 = gbrbm_second(sample_data)
        omse_loss = mse_loss(vog_second, v2)

        gb_second_train_op.zero_grad()
        omse_loss.backward()
        gb_second_train_op.step()

    for _, (data) in enumerate(v2):
        start = time.time()
        data = Variable(
                torch.tensor(data, dtype=torch.float32)
        ).uniform_(0, 1)

        sample_data = torch.normal(mean=data, std=gaussian_std).to(device=device)

        vog_third, v3_e = gbrbm_third(sample_data)
        omse_loss = mse_loss(vog_third, v3_e)
        
        gb_third_train_op.zero_grad()
        omse_loss.backward()
        gb_third_train_op.step()

    svm_X = torch.tensor(v3_e, dtype=torch.float32, device=device)
    svm_Y = torch.tensor(v3_e.view(-1, 107), dtype=torch.float32, device=device)
    N = len(svm_Y)

    perm = torch.randperm(N, device=device)

    for i in range(0, N, BATCH_SIZE):
        correct = float()

        # x = svm_X[perm[i:i + BATCH_SIZE]]
        # y = svm_Y[perm[i:i + BATCH_SIZE]]
        # x = svm_X.to(device=device)
        # y = svm_Y.to(device=device)

        x = torch.tensor(svm_X.clone().detach(), device=device)
        y = torch.tensor(svm_Y.clone().detach(), device=device)

        # Forward
        output = svm_model(x)
        
        # Backward
        svm_optimizer.zero_grad()        
        svm_optimizer.step()

        predicted = torch.tensor(output.data >= 0, dtype=torch.float32)
        print(predicted)

        svm_acc = output.data >= predicted
        
    svm_best_acc = svm_acc
    path = "./mat_svm_model/" + str(epoch) + "_Train_svm_model_acc__" + str(svm_acc) + ".pth"
    torch.save(svm_model, path)

    acc_v = (vog_third >= 0).float()
    acc = get_acc(
        acc_v, v3_e
    ) * 100
    
    if acc > best_acc:
        best_acc = acc    
        path = "./say_cheese/ahh_saveMode_through_" + str(epoch) + "_" + str(acc) + "GBRBM.pth"
        torch.save(gbrbm_third.state_dict(), path)
    output_gb.append(v3_e)

    print("GB-DBN Training loss for {0}th epoch {1}\tEstimate time : {2}\tAcc : {3}\t\tBest Acc : {4}\t\SVM Acc: {5}".format(epoch + 1, omse_loss, time.time() - start, acc, best_acc, svm_acc))
    gc.collect()

RBM START!


C:\Users\HILAB_~1\AppData\Local\Temp/ipykernel_14200/2215458286.py:16: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  data = Variable(torch.tensor(data.uniform_(0, 1), dtype=torch.float32))
c:\Users\HILAB_Labtop_02\anaconda3\lib\site-packages\torch\nn\functional.py:1944: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
C:\Users\HILAB_~1\AppData\Local\Temp/ipykernel_14200/2215458286.py:31: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  torch.tensor(data, dtype=torch.float32)
C:\Users\HILAB_~1\AppData\Local\Temp/ipykernel_14200/2215458286.py:47: UserWarning: To copy construct from a

tensor([1.], device='cuda:0')
GB-DBN Training loss for 1th epoch 0.5412733554840088	Estimate time : 0.01000213623046875	Acc : 64.48598130841121		Best Acc : 64.48598130841121	\SVM Acc: tensor([False], device='cuda:0')
tensor([1.], device='cuda:0')
GB-DBN Training loss for 2th epoch 0.6160750389099121	Estimate time : 0.008995771408081055	Acc : 60.747663551401864		Best Acc : 64.48598130841121	\SVM Acc: tensor([True], device='cuda:0')
tensor([1.], device='cuda:0')
GB-DBN Training loss for 3th epoch 0.7103420495986938	Estimate time : 0.012003421783447266	Acc : 65.42056074766354		Best Acc : 65.42056074766354	\SVM Acc: tensor([False], device='cuda:0')
tensor([1.], device='cuda:0')
GB-DBN Training loss for 4th epoch 0.8715355396270752	Estimate time : 0.01399683952331543	Acc : 57.009345794392516		Best Acc : 65.42056074766354	\SVM Acc: tensor([False], device='cuda:0')
tensor([1.], device='cuda:0')
GB-DBN Training loss for 5th epoch 0.6718328595161438	Estimate time : 0.014999866485595703	Acc : 64

In [20]:
LOAD_PATH = "./say_cheese/ahh_saveMode_through_324_82.2429906542056GBRBM.pth"

load_model = RBM(n_vis=187, n_hid=120, k=K_FOLD, batch=BATCH_SIZE)
load_model.load_state_dict((torch.load(LOAD_PATH)))
load_model.to(device=device)


for i, (data) in enumerate(test_dataloader):
    if i == 939:
        continue
    data = Variable(
            torch.tensor(data, dtype=torch.float32)
    ).uniform_(0, 1)
    
    sample_data = torch.bernoulli(data).view(-1, 107).to(device=device)
    
    # tensor binary
    vog_first, v1 = load_model(sample_data)
    omse_loss = mse_loss(vog_first, v1)
    
    first_train_op.zero_grad()
    first_train_op.step()
    omse_loss.backward()
print("Load model: ", LOAD_PATH)
print(get_acc(vog_first, v1))

C:\Users\HILAB_~1\AppData\Local\Temp/ipykernel_14200/2196614516.py:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  torch.tensor(data, dtype=torch.float32)
c:\Users\HILAB_Labtop_02\anaconda3\lib\site-packages\torch\nn\functional.py:1944: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


Load model:  ./say_cheese/ahh_saveMode_through_324_82.2429906542056GBRBM.pth
0.4897196261682243
